# 1. download places

In [4]:
import duckdb


In [ ]:
with open("/Users/rfitzger/data/census/us-extent.txt") as f:
    boundary = f.read()
places_file = "/Users/rfitzger/data/mep/mep3/input/opportunities/us-places.csv"

In [ ]:
query = f"""
LOAD spatial; -- noqa
LOAD httpfs;  -- noqa
-- Access the data on AWS in this example
SET s3_region='us-west-2';

COPY (
    SELECT
       id,
       version,
    -- We are casting these columns to JSON in order to ensure compatibility with our GeoJSON output.
    -- These conversions may be not necessary for other output formats.
       CAST(names AS JSON) AS names,
       CAST(categories AS JSON) AS categories,
       confidence,
      --  CAST(websites AS JSON) AS websites,
      --  CAST(socials AS JSON) AS socials,
      --  CAST(emails AS JSON) AS emails,
      --  CAST(phones AS JSON) AS phones,
      --  CAST(brand AS JSON) AS brand,
       CAST(addresses AS JSON) AS addresses,
       CAST(sources AS JSON) AS sources,
       geometry AS geometry -- DuckDB v.1.1.0 will autoload this as a `geometry` type
FROM read_parquet('s3://overturemaps-us-west-2/release/2025-02-19.0/theme=places/*/*')
WHERE
   ST_Contains('{boundary}'::GEOMETRY, geometry)
) TO '{places_file}';
"""

In [9]:
print(duckdb.sql(query))

None


# 2. process into opportunity dataset
the expected schema is: ``